# Update (MGP)
Using an Area of Interest (AOI) from the `aoi` SQLite Database, queries the Maxar Geospatial Platform API for data over a given time period, builds the `mgp` table from these results, and then populates the table with these values. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [1]:
# Data Science stack
import geopandas as gpd

# Custom stack
import sys; sys.path.append("../../")
from database import functions
from mgp import security, search

### User defined variables
Some other collections include:
- wv01
- wv02
- wv03-swir
- wv04

In [2]:
dar_id = '24'
collections = ["wv03-vnir"]
start_date = '2023-04-26'
end_date = '2024-04-26'
where_clause = "eo:cloud_cover <= 90 and off_nadir_avg <= 90"
limit = 500
db = "C:/gis/gaia/data/databases/gaia.db"

### Get MGP Token
Hint: ???

In [3]:
token_file = security.fetch_auth()

### Retreve the Area of Interest record from the `aoi` table corresponding to the supplied DAR number

In [4]:
gdf_aoi = functions.get_aoi(db, dar_id)
gdf_aoi

,dar_id,name,sqkm,geometry
0,24,Kodiak Island,416.53,"POLYGON ((-152.71714 57.06081, -152.40833 57.0..."


### Search MGP

In [5]:
gdf_json = gdf_aoi['geometry'].to_json()

response, id_list = search.search(
    collections = collections,
    start = start_date,
    end = end_date,
    where = where_clause,
    geometry = gdf_json,
    limit = limit
)

print("Here are some example ImageIDs your query returned: {}".format(id_list[0:5]))

{
  "datetime": "2023-04-26/2024-04-26",
  "where": "eo:cloud_cover <= 90 and off_nadir_avg <= 90",
  "intersects": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -152.717138,
          57.06081
        ],
        [
          -152.408327,
          57.099546
        ],
        [
          -152.334346,
          56.663026
        ],
        [
          -152.631884,
          56.629267
        ],
        [
          -152.717138,
          57.06081
        ]
      ]
    ]
  },
  "limit": 500,
  "page": 1,
  "collections": [
    "wv03-vnir"
  ]
}
Total of 28 objects returned
Here are some example ImageIDs your query returned: ['1040010095680F00', '1040010095A32700', '1040010093C12900', '104001009399C700', '10400100941FB200']


### Create a GeoDataFrame from query results

In [6]:
gdf = functions.gdf_from_mgp(response, dar_id)
gdf.head()

,id,platform,instruments,gsd,pan_resolution_avg,multi_resolution_avg,datetime,off_nadir,azimuth,sun_azimuth,sun_elevation,bbox,dar_id
0,1040010095680F00,worldview-03,VNIR,0.415387,0.415387,1.659449,2024-04-12T21:01:45.707584Z,31.324845,316.569775,157.668117,40.59462,"POLYGON ((-152.15399 56.61705, -152.15399 57.0...",24
1,1040010095A32700,worldview-03,VNIR,0.399417,0.399417,1.597395,2024-04-12T21:01:31.859214Z,29.463139,301.571675,157.256272,40.570141,"POLYGON ((-152.41008 56.61900, -152.41008 57.0...",24
2,1040010093C12900,worldview-03,VNIR,0.486381,0.486381,1.871329,2024-04-05T20:52:27.715137Z,38.101569,302.732907,154.650647,37.411949,"POLYGON ((-152.34341 56.60253, -152.34341 57.1...",24
3,104001009399C700,worldview-03,VNIR,0.364534,0.364534,1.458094,2024-03-26T21:34:15.475246Z,23.936033,47.818292,167.537767,35.415821,"POLYGON ((-152.24113 56.62537, -152.24113 57.0...",24
4,10400100941FB200,worldview-03,VNIR,0.337946,0.337946,1.351221,2024-03-26T21:34:01.224880Z,17.437244,60.216834,167.21415,35.321595,"POLYGON ((-152.46353 56.64134, -152.46353 57.0...",24


### Insert Maxar Geospatial Platform IDs into `mgp` table

In [7]:
functions.insert_pk(db, 'mgp', gdf)

Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010095680F00
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010095A32700
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010093C12900
Exception: UNIQUE constraint failed: mgp.id was raised for ID 104001009399C700
Exception: UNIQUE constraint failed: mgp.id was raised for ID 10400100941FB200
Exception: UNIQUE constraint failed: mgp.id was raised for ID 10400100936C5600
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010093BBA600
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010092C53200
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010090090300
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010091C88C00
Exception: UNIQUE constraint failed: mgp.id was raised for ID 1040010090ACE000
Exception: UNIQUE constraint failed: mgp.id was raised for ID 104001008E380A00
Exception: UNIQUE constraint failed: mgp.id was rais

### Update `mgp` records from Maxar Geospatial Platform results

In [8]:
columns = gdf.columns[1:]
for i, row in gdf.iterrows():
    iid = row['id']
    print("Updating information for Imagery ID: {}".format(iid))
    for column in columns:
        functions.database_activity(db, 'mgp', iid, column, row[column])

Updating information for Imagery ID: 1040010095680F00
Updating information for Imagery ID: 1040010095A32700
Updating information for Imagery ID: 1040010093C12900
Updating information for Imagery ID: 104001009399C700
Updating information for Imagery ID: 10400100941FB200
Updating information for Imagery ID: 10400100936C5600
Updating information for Imagery ID: 1040010093BBA600
Updating information for Imagery ID: 1040010092C53200
Updating information for Imagery ID: 1040010090090300
Updating information for Imagery ID: 1040010091C88C00
Updating information for Imagery ID: 1040010090ACE000
Updating information for Imagery ID: 104001008E380A00
Updating information for Imagery ID: 104001009094D200
Updating information for Imagery ID: 104001008E3A9100
Updating information for Imagery ID: 10400100913F8800
Updating information for Imagery ID: 104001008C9A2B00
Updating information for Imagery ID: 104001008D9BE500
Updating information for Imagery ID: 104001008E807400
Updating information for Ima

### Select newly inserted AOIs, make a GeoDataFrame for validation

In [9]:
gdf = functions.validate_updates(db, 'mgp', gdf, dar_id)
# Note that the GDF shape matches that from the above
print(gdf.shape)
gdf.head()

(28, 13)


,id,platform,instruments,gsd,pan_resolution_avg,multi_resolution_avg,datetime,off_nadir,azimuth,sun_azimuth,sun_elevation,bbox,geometry
0,1040010095680F00,worldview-03,VNIR,0.415387,0.415387,1.659449,2024-04-12T21:01:45.707584Z,31.324845,316.569775,157.668117,40.594620,b'\x00\x01\x00\x00\x00\x00e\x1b\xb8\x03u\x0ec\...,"POLYGON ((-152.15399 56.61705, -152.15399 57.0..."
1,1040010095A32700,worldview-03,VNIR,0.399417,0.399417,1.597395,2024-04-12T21:01:31.859214Z,29.463139,301.571675,157.256272,40.570141,b'\x00\x01\x00\x00\x00\x00|\x80\xee\xcb\x99\x1...,"POLYGON ((-152.41008 56.61900, -152.41008 57.0..."
2,1040010093C12900,worldview-03,VNIR,0.486381,0.486381,1.871329,2024-04-05T20:52:27.715137Z,38.101569,302.732907,154.650647,37.411949,b'\x00\x01\x00\x00\x00\x00M\xf6\xcf\xd3\x00\x1...,"POLYGON ((-152.34341 56.60253, -152.34341 57.1..."
3,104001009399C700,worldview-03,VNIR,0.364534,0.364534,1.458094,2024-03-26T21:34:15.475246Z,23.936033,47.818292,167.537767,35.415821,b'\x00\x01\x00\x00\x00\x00\x7f\xa0\xdc\xb6\xef...,"POLYGON ((-152.24113 56.62537, -152.24113 57.0..."
4,10400100941FB200,worldview-03,VNIR,0.337946,0.337946,1.351221,2024-03-26T21:34:01.224880Z,17.437244,60.216834,167.214150,35.321595,b'\x00\x01\x00\x00\x00\x00/\xa7\x04\xc4\xa4\x1...,"POLYGON ((-152.46353 56.64134, -152.46353 57.0..."


### Plot Areas of Interest on an Interactive Map

In [10]:
mp = functions.quick_map(gdf, gdf_json)
mp

# End